<h1>RSA 구현</h1>
<h3>202235408 김경훈</h3>

<h2>① 랜덤성 부여를 위한 random import</h2>

In [2]:
import random

<h2>② 기본 함수 구현 - Miller Rabin</h2>

In [3]:
def miller_rabin(n, k) :
    is_composite = False
    is_prime = True
    
    if (n == 2) :
        return False

    if (n == 3) :
        return True

    t = 0
    u = n - 1
    
    while (u % 2 == 0) :
        t += 1
        u //= 2
        
    for _ in range(0, k):
        a = random.randrange(2, n - 2)
        x = pow(a, u, n)
        
        for _ in range(0, t) :
            y = pow(x, 2, n)
            
            if (y == 1 and x != 1 and x != n - 1) :
                return is_composite
            
            x = y
            
        if (x != 1) :
            return is_composite
        
    return is_prime

<h2>③ 기본 함수 구현 - GCD</h2>

In [4]:
def gcd(m, n):
    if n == 0:
        return m
    
    mod = m % n
    
    if mod != 0:
        m = n
        n = mod
        
        return gcd(m, n)
    
    else:
        return n

<h2>③ 기본 함수 구현 - ExtendedEuclidean</h2>

In [5]:
def extended_euclidean(m, n) :
    if (n == 0) :
        return 1, 0
        
    tmp_s, tmp_t = extended_euclidean(n, m % n)
    
    s = tmp_t
    t = tmp_s - tmp_t * (m // n)
    
    return s, t

<h2>④ 기본 함수 구현 - 지수 계산 함수 (교재 9장의 알고리즘)</h2>

In [6]:
def exponent_cal(a, b, n) :
    bk = bin(b)[2:len(bin(b))]
    bin_b = []
    k = len(bk)
    c = 0
    f = 1
    
    for i in bk :
        bin_b.append(int(i))
    
    bin_b.reverse()
    
    for i in range(k, 0, -1) :
        c *= 2
        f = (f * f) % n
        
        if (bin_b[i - 1] == 1) :
            c += 1
            f = (f * a) % n
            
    return f

<h2>⑤ 기본 함수 구현 - 중국인의 나머지 정리를 이용한 지수 계산 함수</h2>

In [27]:
def exponent_cal_crt(a, b, n, prime1, prime2) :
    if (n == 1) :
        return 0
    
    p = prime1
    q = prime2
    
    cal1 = exponent_cal(a, b, p)
    cal2 = exponent_cal(a, b, q)
    
    result = (cal1 * q * pow(q, -1, p) + cal2 * p * pow(p, -1, q)) % n
    
    return result

<h2>⑥ 소수 생성 함수</h2>

In [8]:
def make_prime() :
    while(True) :
        prime = random.randint(1, 9999)
        
        if (miller_rabin(prime, 5) == 1) :
            break
        
        else :
            continue
            
    return prime

<h2>⑦ 키 생성 함수</h2>

In [9]:
def make_key(p, q) :
    n = p * q
     
    while(True) :
        e = random.randint(1, 65537)
        
        if (e % 2 == 1) :
            break
        
        else :
            continue 
        
    print("GCD(e, p-1) : ", gcd(e, p-1))
    print("GCD(e, q-1) : ", gcd(e, q-1))
    
    if (gcd(e, p - 1) != 1 and gcd(e, q - 1) != 1) :
        print("Error : please make other prime for p or q")
        
        exit()
    
    result = extended_euclidean(e, (p - 1) * (q - 1))
    d = result[0]
    
    if (d < 0) :
        d += (p - 1) * (q - 1)
    
    public_key = (e, n)
    private_key = (d, p, q)
    
    return public_key, private_key

<h2>⑧ 메시지 생성 함수</h2>

In [30]:
def make_msg() :
    M = 0
    msg = input("Enter your msg : ")
    
    for m in msg :
        M += ord(m)
        M += ord("/")
        
    return M

<h2>⑨ 메시지 암호화 함수</h2>

In [11]:
def msg_encrypt(msg, public_key) :
    e = public_key[0]
    n = public_key[1]
    
    cipher = exponent_cal(msg, e, n)
    
    return cipher

<h2>⑩ - ⒧ 메시지 복호화 함수 - 일반</h2>

In [12]:
def msg_decrypt(cipher, public_key, private_key) :
    n = public_key[1]
    d = private_key[0]
    
    dmsg = exponent_cal(cipher, d, n)
    
    return dmsg

<h2>⑩ - ⑵ 메시지 복호화 함수 - 중국인의 나머지 정리 이용</h2>

In [26]:
def msg_decrypt_crt(cipher, public_key, private_key, p, q) :
    n = public_key[1]
    d = private_key[0]
    
    dmsg = exponent_cal_crt(cipher, d, n, p, q)
    
    return dmsg

<h2>⑪ 평문과 복호화된 평문 비교 출력 함수</h2>

In [38]:
def compare_msg(msg, dmsg) :  
    print(f"Plain Text (number value) : {msg}")
    print(f"Decrypted Plain Text (number value) : {dmsg}")

<h2>■ 소수 p, q 생성</h2>

In [15]:
p = make_prime()
q = make_prime()

print(f"p : {p}, q : {q}")

p : 67, q : 9781


<h2>■ 임의의 메세지 생성</h2>

In [31]:
msg = make_msg()

print(msg)

703


<h2>■ 키 생성</h2>
<h3>- 키 생성 과정에서 gcd(e, p - 1) 혹은 gcd(e, q - 1)의 값이 1이 아닌 경우 프로그램 종료 후 새로운 소수를 생성하라는 안내 문구 출력 </h3>

In [32]:
key_set = make_key(p, q)

public_key = key_set[0]
private_key = key_set[1]

print(public_key)
print(private_key)

GCD(e, p-1) :  1
GCD(e, q-1) :  1
(39883, 655327)
(433627, 67, 9781)


<h2>■ 메시지 암호화</h2>

In [33]:
encryption = msg_encrypt(msg, public_key)

print(encryption)

349173


<h2>■ 메시지 복호화 (중국인의 나머지 정리 이용 X)</h2>

In [34]:
decryption = msg_decrypt(encryption, public_key, private_key)

print(decryption)

703


<h2>■ 메시지 복호화 (중국인의 나머지 정리 이용)</h2>

In [35]:
decryption_crt = msg_decrypt_crt(encryption, public_key, private_key, p, q)

print(decryption_crt)

703


<h2>■ 평문과 복호화된 평문 비교</h2>

In [42]:
compare_msg(msg, decryption)

Plain Text (number value) : 703
Decrypted Plain Text (number value) : 703
